# IIC2233 2024-2. Semana 16: Tópicos Avanzados 2 (Sistemas recomendadores)

Demo práctica para recomendar anime usando 3 algoritmos distintos:
- _Most Popular_
- Basado en Contenidos con KNN.
- Filtrado Colaborativo con KNN.
- Híbrido (Colaborativo y Contenido) con Redes Neuronales

Demo creada por el profesor [Hernán Valdivieso](https://hernan4444.github.io/).

Los datos utilizados son de este [dataset](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020) creado por el mismo profesor con información de [MyAnimeList](https://myanimelist.net/).

# Parte 1 - Datos

## Descargar datos

Utilizamos la librería [`gdown`](https://pypi.org/project/gdown/) para descargar 2 datasets:
1. El dataset de [anime](https://drive.google.com/file/d/1KhLqWalpy4YmcGbu4av1qx40yJoaKJxm/view?usp=drive_link).
2. El dataset con las [interacciones usuario-anime-rating](https://drive.google.com/file/d/1C3h0bM11cxKxobQrks-Grgttgb7yvagK/view?usp=drive_link). Se ocupó el _dataset_ que solo tiene animes que fueron completamente visto a la fecha de obtener la información.

In [ ]:
!gdown 1KhLqWalpy4YmcGbu4av1qx40yJoaKJxm
!gdown 1C3h0bM11cxKxobQrks-Grgttgb7yvagK

# Extra: modelo pre-entrenado para el última modelo que veamos
# spoiler: no es tan bueno.
!gdown 1P-iU7oYbmithNg0hSU2woTk8XkGmZk8v

Además, para el último algoritmo queremos recomendar en función de un usuario real (el profesor en este caso). Por lo cual, se recomienda:
1. Tener cuenta en MyAnimeList
2. [Visitar este enlace](https://myanimelist.net/panel.php?go=export)
3. Descargar lista de animes
4. Cambiar nombre de archivo por `animelist.xml.gz`
5. Posicionar el archivo junto a este código.

En caso de no tener cuenta, no hay que preocuparse. Podemos crear manualmente esto.

## Cargar datos

Empezamos cargando ambos dataset (anime e interacciones) y observamos su contenido.

In [ ]:
import pandas as pd
from collections import Counter

df_anime = pd.read_csv('anime_actualizado.csv')
df_rating = pd.read_csv('rating_complete.csv')

print(df_anime.shape)
df_anime.head()

In [ ]:
df_anime.Type.unique()

In [ ]:
df_anime = df_anime[df_anime.Score != "Unknown"]

df_anime.loc[:, "Score"] = df_anime.Score.astype(float)
df_anime = df_anime[df_anime.Score > 5]

tipos_aceptados = set(["TV", "Movie", "OVA"])
df_anime = df_anime[df_anime.Type.isin(tipos_aceptados)]

df_anime = df_anime.reset_index(drop=True)
print(df_anime.shape)
df_anime.head()

In [ ]:
print(df_rating.shape)
df_rating.head()

Solo de curioso, vamos a ver cuantas interacciones por calificación hay.

In [ ]:
import matplotlib.pyplot as plt

ax = df_rating.groupby("rating").size().plot(kind="bar")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
plt.show()

Vamos a reducir la cantidad para no considerar animes que no tenemos metadata (eliminará aprox 7 millones de datos).

In [ ]:
Id_aceptados = set(df_anime.MAL_ID.unique())
df_rating = df_rating[df_rating.anime_id.isin(Id_aceptados)]
print(df_rating.shape)


Sigue siendo mucho, mejor nos quedamos con un random de 1 millon para usar despues.

In [ ]:
df_rating = df_rating.sample(1000000).reset_index(drop=True)
print(df_rating.shape)
df_rating.head()

Verifiquemos si la distribución de ratings sigue igual

In [ ]:
import matplotlib.pyplot as plt

ax = df_rating.groupby("rating").size().plot(kind="bar")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
plt.show()

In [ ]:
# Obtener las frecuencias de las clases (ratings)
class_counts = Counter(df_rating["rating"])

# Calcular pesos inversamente proporcionales a las frecuencias
class_weights = {rating: 1.0 / count for rating, count in class_counts.items()}
df_rating["weight"] = df_rating["rating"].map(class_weights)
df_rating.head()

# Modelos

Ahora vamos a construir cada modelo de recomendación. Partiendo del más fácil en implementar al más complejo.

## Most Popular

Este algoritmo solo considera la cantidad de _views_ del anime. Así que solo debemos contar cuanta gente lo evaluó para saber cuál es el más visto. El _rating_ del anime nunca es considerado.

In [ ]:
import pandas as pd

def most_popular_recommendations(df_rating, n_recommendations=10):
    # Contar la cantidad de usuarios que registraron cada anime
    anime_views_df = df_rating.groupby('anime_id')['user_id'].count().reset_index()
    anime_views_df.rename(columns={'user_id': 'views_count'}, inplace=True)

    # Ordenar los animes por la cantidad de registros (los más populares primero)
    most_popular_animes = anime_views_df.sort_values('views_count', ascending=False)

    # Obtener los n_recommendations animes más populares
    recommendations = most_popular_animes.head(n_recommendations)

    data_final = []
    for i, row in recommendations.iterrows():
        # Obtener la información
        anime_info = df_anime[df_anime['MAL_ID'] == row.anime_id].iloc[0]
        data = {
            "MAL_ID": "",
            "Name": "",
            "views": row.views_count
        }
        data.update(anime_info.to_dict())
        data_final.append(data)

    df_final = pd.DataFrame(data_final)
    return df_final

# Ejemplo de uso
recommendations = most_popular_recommendations(df_rating, 5)

recommendations

## Filtrado basado en Contenido

Ahora vamos a ir con un algoritmo un poco más complejo. Para esto, se hará lo siguiente:
1. Cada anime se convertirá en un _embedding_ que represente su información. Para esto vamos a pre-procesar todos los datos para que sean numéricos.
2. Vamos a comprimir el _embedding_ para que sea de un tamaño más chico.
3. Entrena 2 modelos (k-nearest neighbors - KNN) para determinar, dado 1 anime, cuáles son los más cercanos a él. Un modelo ocupará el _embedding_ original mientras otro ocupará la versión comprimida.
4. Finalmente, dado 1 anime de la base de datos, vamos a recomendar 10 anime más.


### Librerías necesarias

Primero hacemos _import_ de las librerías necesarias para esta parte:

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

### Preparar dataset

Nos quedamos solo con las columnas que usaremos en la recomendación basada en contenido.

In [ ]:
columnas_necesarias = ["MAL_ID", "Name", "Score", "Genres", "Type",
                       "Episodes", "Premiered", "Studios", "Source", "Rating",
                       "Members"]

df_contenido = df_anime.loc[:, columnas_necesarias]
df_contenido.head(5)

Procesamos algunas columnas. En particular,
* Generos y Estudio, como son atributos con muliples valores, los vamos a transformar en un lista.
* Puntaje y Episodios, puede existir animes con valor `"Unknown"` que vamos a transformar en 0.

In [ ]:
def process_multilabel(series):
    series = series.split(", ")
    if "Unknown" in series:
        series.remove("Unknown")
    return series

# Atributos con múltiples valores
df_contenido["Genres"] = df_contenido["Genres"].map(process_multilabel)
df_contenido["Studios"] = df_contenido["Studios"].map(process_multilabel)

# Atributo con Unknown
df_contenido["Episodes"] = df_contenido["Episodes"].replace("Unknown", 0).astype(int)

df_contenido.head()

Ahora, vamos a transformar todas las columnas con información a algo numérico. Para esto:
1. Usaremos `one-hot-encoding` para todas las columnas categóricas. Es decir, si tengo `["A", "B"]`, se crearán 2 columnas: `"Columna_A"` y `"Columna_B"` donde habrá un 1 si el anime tiene ese valor o un 0 si no lo tiene.
2. Todas las columnas numéricas serán escaladas para que estén en un rango de 0-1.

In [ ]:
# Función para procesar categorías
def preprocessing_category(df, column, is_multilabel=False):
    lb = LabelBinarizer()
    if is_multilabel:
        lb = MultiLabelBinarizer()

    expandedLabelData = lb.fit_transform(df[column])
    labelClasses = lb.classes_

    # Create a pandas.DataFrame from our output
    category_df = pd.DataFrame(expandedLabelData, columns=labelClasses)
    del df[column]
    return pd.concat([df, category_df], axis=1)

anime_metadata = df_contenido.copy()

# Procesar categorías
anime_metadata = preprocessing_category(anime_metadata, "Type")
anime_metadata = preprocessing_category(anime_metadata, "Premiered")
anime_metadata = preprocessing_category(anime_metadata, "Studios", is_multilabel=True)
anime_metadata = preprocessing_category(anime_metadata, "Genres", is_multilabel=True)
anime_metadata = preprocessing_category(anime_metadata, "Source")
anime_metadata = preprocessing_category(anime_metadata, "Rating")

# Quedarme con las
ID_NAME = anime_metadata[["MAL_ID", "Name"]]

del anime_metadata["MAL_ID"]
del anime_metadata["Name"]
del anime_metadata["Unknown"]

scaled_data = MinMaxScaler().fit_transform(anime_metadata[["Score", "Episodes", "Members"]])
anime_metadata[["Score", "Episodes", "Members"]] = scaled_data

anime_metadata_values = anime_metadata.values

print(anime_metadata.shape)
anime_metadata.head()

Vamos a crear un nuevo dataset donde vamos a "comprimir" las 910 columnas en solo 200 columnas

In [ ]:
svd = TruncatedSVD(n_components=200)
anime_svd_values = svd.fit_transform(anime_metadata_values)

# Crear un nuevo DataFrame con las coordenadas
anime_svd_df = pd.DataFrame(data=anime_svd_values)

# Concatenar con el dataframe original para tener la información del nombre
anime_svd_df = pd.concat([ID_NAME, anime_svd_df], axis=1)
anime_svd_df.head()

### Entrenar modelo

Creamos 2 modelos, uno que ocupe la información original y otro que ocupe la comprimida

In [ ]:
# Crear modelo KNN con distancia coseno para encontrar vectores cercanos.
model_knn_metadata = NearestNeighbors(metric='cosine', n_neighbors=10)
model_knn_svd = NearestNeighbors(metric='cosine', n_neighbors=10)

model_knn_metadata.fit(csr_matrix(anime_metadata_values))
model_knn_svd.fit(csr_matrix(anime_svd_values))

### Recomendar

In [ ]:
# Crear función que ocupa el modelo para entregar recomendación dado 1 anime
def get_recommended(modelo, anime_vector, vecinos=10):
    distances, indices = modelo.kneighbors(anime_vector, n_neighbors=vecinos)
    indices, distances = indices.flatten(), distances.flatten()
    result = []

    # Partimos en 1 porque la posición 0 siempre será el mismo anime qe le solicitamos
    for i in range(1, len(distances.flatten())):
        data = {"MAL_ID": "", "Name": "", "distancia": distances[i]}
        anime_recomendado = df_contenido.iloc[indices[i]].to_dict()
        data.update(anime_recomendado)
        result.append(data)

    return pd.DataFrame(result)

Escogemos 1 anime según su ID en MyAnimelist

In [ ]:
# Kimi no Na wa --> MAL_ID == 32281
indice = ID_NAME[ID_NAME.MAL_ID == 32281].index[0]

df_contenido.iloc[[indice]]

Recomendamos usando nuestro modelo que ocupa metadata

In [ ]:
anime_vector = anime_metadata_values[indice,:].reshape(1, -1)
get_recommended(model_knn_metadata, anime_vector, 10)

Recomendamos usando nuestro modelo que ocupa la información comprimida

In [ ]:
anime_vector = anime_svd_values[indice,:].reshape(1, -1)
get_recommended(model_knn_svd, anime_vector, 10)

## Filtrado Colaborativo

Vamos a realizar el mismo tipo de recomendación, pero usando otro vector de entrada:

1. Cada anime se convertirá en un _embedding_ que represente su información. Para esto se ocuparán las interacciones.
3. Entrenar 1 modelos (k-nearest neighbors - KNN) para determinar, dado 1 anime, cuáles son los más cercanos a él.
4. Finalmente, dado 1 anime de la base de datos, vamos a recomendar 10 anime más.


### Librerías Necesarias

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [ ]:
df_rating_2 = pd.read_csv('rating_complete.csv')
Id_aceptados = set(df_anime.MAL_ID.unique())
df_rating_2 = df_rating_2[df_rating_2.anime_id.isin(Id_aceptados)]
print(df_rating_2.shape)

In [ ]:
count_user = df_rating_2['user_id'].value_counts()
count_user = set(count_user[count_user > 300].index)

count_anime = df_rating_2['anime_id'].value_counts()
count_anime = set(count_anime[count_anime > 500].index)

In [ ]:
rating_data = df_rating_2[df_rating_2['user_id'].isin(count_user)]
rating_data = rating_data[rating_data['anime_id'].isin(count_anime)].reset_index(drop=True)

del count_user
del count_anime

cantidad_usuario = len(rating_data.user_id.unique())
cantidad_anime = len(rating_data.anime_id.unique())

print(f"Cantidad de usuarios: {cantidad_usuario}")
print(f"Cantidad de animes: {cantidad_anime}")
print(f"Cantidad interacciones: {len(rating_data)}")
print(f"Tamaño matriz: {cantidad_usuario * cantidad_anime}")

In [ ]:
rating_matrix = rating_data.pivot(index='anime_id', columns='user_id', values='rating')
rating_matrix.fillna(0, inplace=True)

print(rating_matrix.shape)
rating_matrix.head()

### Entrenamiento

In [ ]:
piviot_table_matrix = csr_matrix(rating_matrix)
piviot_table_matrix

In [ ]:
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

### Recomendación

In [ ]:
MAL = 32281
MAL = 16498
MAL = 1575
df_anime[df_anime.MAL_ID == MAL]

In [ ]:
vector = rating_matrix.loc[MAL].values
vector = vector.reshape(1, -1)

distance, suggestions = model.kneighbors(vector, n_neighbors=6)
distance = distance.flatten()
suggestions = suggestions.flatten()

MAL_ID = []
for i in range(0, len(distance)):
    matrix_index = suggestions[i]
    anime_index = rating_matrix.index[matrix_index]
    print('{0}: {1}, with distance of {2}:'.format(i, anime_index, distance[i]))
    MAL_ID.append(df_anime[df_anime.MAL_ID == anime_index].iloc[0])

pd.DataFrame(MAL_ID)

Eliminar variables para liberar ram

In [ ]:
del rating_matrix
del df_rating_2
del rating_data
del model
del piviot_table_matrix

## Modelo híbrido

Ahora vamos a utilizar el modelo más complejo, uno con redes neuronales. El enfoque de este modelo será predecir el _rating_ que un usuario le dará a un anime. De este modo, la recomendación se basará en dar los animes que se espera que tengan mayor _rating_.

#### Librerías necesarias

Primero hacemos _import_ de las librerías necesarias para esta parte:

In [ ]:
from IPython.display import display
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np
from torch.utils.data import WeightedRandomSampler
import os
from tqdm import tqdm
from google.colab import files

### Preprocesamiento de los datos

In [ ]:
# Función para procesar categorías
def preprocessing_category(df, column, is_multilabel=False):
    lb = LabelBinarizer()
    if is_multilabel:
        lb = MultiLabelBinarizer()

    expandedLabelData = lb.fit_transform(df[column])
    labelClasses = lb.classes_

    # Create a pandas.DataFrame from our output
    category_df = pd.DataFrame(expandedLabelData, columns=labelClasses)
    del df[column]
    return pd.concat([df, category_df], axis=1)

anime_metadata = df_contenido.copy()

# Procesar categorías
anime_metadata = preprocessing_category(anime_metadata, "Type")
anime_metadata = preprocessing_category(anime_metadata, "Premiered")
anime_metadata = preprocessing_category(anime_metadata, "Studios", is_multilabel=True)
anime_metadata = preprocessing_category(anime_metadata, "Genres", is_multilabel=True)
anime_metadata = preprocessing_category(anime_metadata, "Source")
anime_metadata = preprocessing_category(anime_metadata, "Rating")

del anime_metadata["MAL_ID"]
del anime_metadata["Name"]
del anime_metadata["Unknown"]

In [ ]:
print("Anime")
display(df_anime.head(2))

print("\nRatings")
display(df_rating.head(2))

print("\nMetadata procesada")
display(anime_metadata.head(2))

In [ ]:
COLUMNAS_METADATA = 910

# Dado el ID de un anime, obtengo su vector de metadata
item_metadata = {}
values_metadata = anime_metadata.values.tolist()
anime_id = df_anime["MAL_ID"].values.tolist()
for i, id_ in enumerate(anime_id):
    item_metadata[id_] = values_metadata[i]

del values_metadata
del anime_id

# Dado el ID de un usuario, obtengo la lista de animes que vio
animes_per_user = df_rating[["user_id", "anime_id"]].groupby("user_id")['anime_id'].apply(list)

print(item_metadata[1][:9])
print(item_metadata[5][:9])

usuario = list(animes_per_user.keys())[0]
df_anime.loc[df_anime["MAL_ID"].isin(animes_per_user[usuario])]

### Confección del modelo


In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.constant_(m.bias, 0.01)

class MyAnimeListRecomendador(nn.Module):
    def __init__(self, metadata_dim, embedding_dim, hidden_dim, dropout_rate=0.2):
        super().__init__()

        # Capa para obtener los embeddings de los ítems a partir de sus metadatos
        self.item_metadata_fc = nn.Linear(metadata_dim, embedding_dim)

        # Capa para obtener los embeddings de los ítems a partir de sus metadatos
        self.user_metadata_fc = nn.Linear(metadata_dim, embedding_dim)

        # Capas ocultas para modelar la interacción usuario-ítem
        self.hidden_layer1 = nn.Linear(embedding_dim*2, hidden_dim)
        self.hidden_layer2 = nn.Linear(hidden_dim, hidden_dim*2)

        # Función de activación y regularización
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim*2, 1)

        self.apply(initialize_weights)

    def forward(self, user_metadata, item_metadata):
        """
        Args:
        - user_item_metadata: Tensor de metadatos de los ítems que el usuario ha consumido (batch_size, user_embedding_dim)
        - item_metadata: Tensor de metadatos del ítem a predecir (batch_size, metadata_dim)
        """
        # Obtener embedding para el ítem actual
        item_embedding = self.item_metadata_fc(item_metadata)

        # Obtener embedding para el usuario actual
        user_embedding = self.user_metadata_fc(user_metadata)

        # Concatenar los embeddings del usuario y del ítem
        interaction = torch.cat([user_embedding, item_embedding], dim=-1)

        # Pasar la concatenación por las capas ocultas
        x = self.hidden_layer1(interaction)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden_layer2(x)

        # Predicción final
        output = self.fc(x).squeeze()
        return output

### Entrenamiento del modelo

In [ ]:
# Función para sumar cada metadata de los videos visto por un usaurio
def obtener_embedding_usuario(item_metadata, animes_visto_usuario):
    user_metadata = np.zeros(COLUMNAS_METADATA)
    for anime_id in animes_visto_usuario:
        user_metadata += np.array(item_metadata[anime_id])

    return user_metadata

# Dataset personalizado para el entrenamiento, para cada tripleta de
# (usuario, anime, rating) genera los datos necesarios para el modelo.
class InteractionDataset(Dataset):
    def __init__(self, interactions, item_metadata, animes_per_user):
        self.interactions = interactions
        self.item_metadata = item_metadata
        self.anime_per_user = animes_per_user

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        user_id, item_id, rating, _ = self.interactions.iloc[idx]
        animes_visto_usuario = self.anime_per_user[user_id]

        user_metadata = obtener_embedding_usuario(self.item_metadata,
                                                  animes_visto_usuario)

        user_metadata -= np.array(self.item_metadata[item_id])
        if len(animes_visto_usuario) > 1:
            user_metadata = user_metadata/(len(animes_visto_usuario) - 1)

        return [
            torch.tensor(user_metadata, dtype=torch.float32),
            torch.tensor(self.item_metadata[item_id], dtype=torch.float32),
            torch.tensor(rating, dtype=torch.float32)
        ]

In [ ]:
def entrenar(modelo, batch_size, epochs, learning_rate, max_samples, save):
    # Dataset de entrenamiento
    dataset = InteractionDataset(df_rating, item_metadata, animes_per_user)

    # Elementos para el entrenamiento
    criterion = nn.MSELoss()  # Usamos MSE para la predicción de ratings
    optimizer = optim.Adam(modelo.parameters(), lr=learning_rate)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Empezamos entrenamiento
    modelo.train()
    modelo = modelo.to(device)

    # Crear un sampler que tome muestras balanceadas
    sampler = WeightedRandomSampler(df_rating["weight"], num_samples=max_samples, replacement=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    for epoch in range(epochs):
        total_loss = 0
        print(f"Época {epoch + 1}")
        for user_embedding, item_metadata_embedding, ratings in tqdm(dataloader):
            user_embedding = user_embedding.to(device)
            item_metadata_embedding = item_metadata_embedding.to(device)
            ratings = ratings.to(device)

            # Obtener predicciones
            preds = modelo(user_embedding, item_metadata_embedding)
            #print(user_embedding.sum(), item_metadata_embedding.sum(), ratings)

            # Calcular error de la predicción
            loss = criterion(preds, ratings)

            # Ajustar parámetros del modelo en función del error
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print()
        print(f"\tLoss Promedio: {total_loss / len(dataloader)}")
        if save:
            torch.save(modelo.state_dict(), 'MyAnimeListRecomendador.pth')

Entrenamiento del modelo, versión clases

In [ ]:
# Constantes
batch_size = 10
epochs = 3
learning_rate = 0.01
max_samples = 500

modelo = MyAnimeListRecomendador(COLUMNAS_METADATA, embedding_dim=512, hidden_dim=512)
entrenar(modelo, batch_size, epochs, learning_rate, max_samples, save=False)

Entrenamiento del modelo, versión extendida (para dejarlo en casa corriendo).

**Importante** Según las contantes ocupadas, esto puede consumir muchas hroas.

In [ ]:
batch_size = 10
epochs = 1
learning_rate = 0.01
max_samples = 20

modelo = MyAnimeListRecomendador(COLUMNAS_METADATA, embedding_dim=512,
                                 hidden_dim=512)

if os.path.exists('MyAnimeListRecomendador.pth'):
    print("Cargando pesos")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pesos = torch.load('MyAnimeListRecomendador.pth',
                       map_location=device, weights_only=True)
    modelo.load_state_dict(pesos)

entrenar(modelo, batch_size, epochs, learning_rate, max_samples, save=False)

### Recomendación

Aquí vamos a usar 2 modelos. El que entrenamos recientemente y otro que fue entrenado previamente por mucho más tiempo.

Para ambos, vamos a probar con:

* El usuario cargado al inicio de la demo (cuenta MyAnimeList).
* Un usuario totalmente nuevo (no ha visto anime).
* Un usuario creado ahora con algunos animes que diga el público.

In [ ]:
COLUMNAS_METADATA = 910

modelo_preentrenado = MyAnimeListRecomendador(COLUMNAS_METADATA,
                                embedding_dim=512,
                                hidden_dim=512)

# Cargamos los pesos (en caso de existir)
if os.path.exists('MyAnimeListRecomendador.pth'):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pesos = torch.load('MyAnimeListRecomendador.pth', map_location=device, weights_only=True)
    modelo_preentrenado.load_state_dict(pesos)

In [ ]:
import pandas as pd
import numpy as np
import torch

def obtener_embedding_usuario_v2(item_metadata, animes_visto_usuario):
    user_metadata = np.zeros(COLUMNAS_METADATA)
    count = 0
    for anime_id in animes_visto_usuario:
        if anime_id in item_metadata:
            user_metadata += np.array(item_metadata[anime_id])
            count += 1
    return user_metadata, count

def recommend_animes(model, item_metadata, user_animes, df_anime, top_n=10):
    # Create user embedding
    user_embedding, count = obtener_embedding_usuario_v2(item_metadata, user_animes)
    if count > 0:
        user_embedding /= count

    user_embedding = torch.tensor(user_embedding, dtype=torch.float32)

    # Get all anime IDs not in the user's list
    all_anime_ids = set(df_anime["MAL_ID"])
    user_anime_ids = set(user_animes)
    anime_ids_to_predict = list(all_anime_ids - user_anime_ids)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    user_embedding = user_embedding.to(device)

    predictions = []
    with torch.no_grad():
        for anime_id in anime_ids_to_predict:
            item_metadata_embedding = torch.tensor(item_metadata[anime_id], dtype=torch.float32)
            item_metadata_embedding = item_metadata_embedding.to(device)

            prediction = model(user_embedding, item_metadata_embedding).item()
            predictions.append((anime_id, prediction))

    # Sort by predicted rating and get top N recommendations
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = predictions[:top_n]
    bottom_recommendations = predictions[-5:]

    animes_ids, ratings = zip(*(top_recommendations + bottom_recommendations))

    recommended_animes = df_anime[df_anime["MAL_ID"].isin(animes_ids)]
    recommended_animes.loc[:, "Rating"] = ratings
    columnas_deseadas = ["MAL_ID", "Name", "Genres", "Score", "Type", "Rating", "Premiered"]

    return recommended_animes.loc[:, columnas_deseadas]

In [ ]:
df_animes_recomendar = df_anime.copy()

# Limitar posibles animes a recomendar
df_animes_recomendar = df_animes_recomendar[df_animes_recomendar.Score > 8]
df_animes_recomendar = df_animes_recomendar[df_animes_recomendar.Type.isin(["TV"])]
df_animes_recomendar = df_animes_recomendar[~df_animes_recomendar['Genres'].str.contains('Unknown', na=False)]
df_animes_recomendar = df_animes_recomendar[~df_animes_recomendar['Genres'].str.contains('Hentai', na=False)]
df_animes_recomendar = df_animes_recomendar.reset_index(drop=True)

print(df_animes_recomendar.shape)
df_animes_recomendar.loc[:, ["MAL_ID", "Name", "Genres", "Score", "Type", "Premiered"]].head(10)

#### Caso 1 - Usuario de MyAnimelist

In [ ]:
# Abrir el archivo anime.xml.gz con pandas
df_anime_user = pd.read_xml('animelist.xml.gz',xpath=".//anime")
user_anime_list = df_anime_user.loc[:, "series_animedb_id"].tolist()

recommend_animes(modelo, item_metadata, user_anime_list, df_animes_recomendar)

In [ ]:
recommend_animes(modelo_preentrenado, item_metadata, user_anime_list, df_animes_recomendar)

#### Caso 2 - Usuario 100% nuevo

In [ ]:
recommend_animes(modelo, item_metadata, [], df_animes_recomendar)

In [ ]:
recommend_animes(modelo_preentrenado, item_metadata, [], df_animes_recomendar)

#### Caso 3 - Usuario creado ahora

In [ ]:
df_animes_recomendar.loc[:, ["MAL_ID", "Name", "Genres", "Score"]].sort_values(by="Score", ascending=False).head(10)

In [ ]:
series_animedb_id = [52991, 5114]
recommend_animes(modelo, item_metadata, series_animedb_id, df_animes_recomendar)

In [ ]:
recommend_animes(modelo_preentrenado, item_metadata, series_animedb_id, df_animes_recomendar)